In [95]:
from pyspark.sql import HiveContext
from pyspark.sql.functions import lower, col, lit, regexp_replace, trim, substring, when, expr, udf, count
from pyspark.sql.types import IntegerType, StringType

In [101]:
def remove_after_hiphen(team_name):
    if team_name.startswith('atletico') or team_name.startswith('atl')  or team_name.startswith('Atl') or team_name.startswith('Ath'):
        return team_name
    else:
        return team_name.split('-', 1)[0]

### Como capturar os dados direto do Cartola FC?

In [117]:
# Programaticamente, através de requisições HTTP, conforme exemplo abaixo.
import requests
import json

url = "https://api.cartolafc.globo.com/partidas/1"

response = requests.get(url)

with open('test_cartola_outputfile.json', 'wb') as outf:
    outf.write(response.content)

data = json.load(open('test_cartola_outputfile.json'))


In [1]:
print(sc)

<SparkContext master=local[*] appName=pyspark-shell>


In [2]:
#df = sqlContext.read.json(json.dumps(data))

df = spark.read.json(sc.parallelize([data]))
df.schema

NameError: name 'data' is not defined

In [3]:
sqlContext = HiveContext(sc)

NameError: name 'HiveContext' is not defined

In [4]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [5]:
print(type(data))

NameError: name 'data' is not defined

In [ ]:
spark

## Colocando os arquivos da pasta cartola no HDFS:
### hadoop fs -put /cartola/* /cartola

### Validando os arquivos csv:

In [4]:
partidas_2014_df = spark.read.csv("/cartola/data/2014/2014_partidas.csv", header=True)
partidas_2014_df.schema

StructType(List(StructField(_c0,StringType,true),StructField(game,StringType,true),StructField(round,StringType,true),StructField(date,StringType,true),StructField(home_team,StringType,true),StructField(score,StringType,true),StructField(away_team,StringType,true),StructField(arena,StringType,true),StructField(X,StringType,true)))

In [5]:
partidas_2014_df.show(5)

+---+----+-----+------------------+---------------+-----+----------------+--------------------+----+
|_c0|game|round|              date|      home_team|score|       away_team|               arena|   X|
+---+----+-----+------------------+---------------+-----+----------------+--------------------+----+
|  1|   1|    1|20/04/2014 - 18:30|  Flamengo - RJ|0 x 0|      Goiás - GO|Mané Garrincha - ...|null|
|  2|   2|    1|19/04/2014 - 18:30|Fluminense - RJ|3 x 0|Figueirense - SC|Maracanã - Rio de...|null|
|  3|   3|    1|20/04/2014 - 16:00| São Paulo - SP|3 x 0|   Botafogo - RJ|Morumbi - Sao Pau...|null|
|  4|   4|    1|20/04/2014 - 18:30|    Santos - SP|1 x 1|      Sport - PE|Vila Belmiro - Sa...|null|
|  5|   5|    1|20/04/2014 - 16:00|  Atletico - PR|1 x 0|     Grêmio - RS|Orlando Scarpelli...|null|
+---+----+-----+------------------+---------------+-----+----------------+--------------------+----+
only showing top 5 rows



In [87]:
remove_hiphen_udf = udf(remove_after_hiphen, StringType())

In [91]:
# Criar nome do time com a string antes do Hífen
partidas_2014_ct = partidas_2014_df.withColumn('away_team', remove_hiphen_udf(partidas_2014_df['away_team']))

partidas_2014_ct = partidas_2014_ct.withColumn('away_team', lower(col('away_team')))

partidas_2014_ct = partidas_2014_ct.withColumn('away_team', trim(col('away_team')))

partidas_2014_ct = partidas_2014_ct.withColumn('home_team', remove_hiphen_udf(partidas_2014_ct['home_team']))

partidas_2014_ct = partidas_2014_ct.withColumn('home_team', lower(col('home_team')))

partidas_2014_ct = partidas_2014_ct.withColumn('home_team', trim(col('home_team')))

partidas_2014_ct = partidas_2014_ct.withColumn('home_score', substring('score', 1,1))\

partidas_2014_ct = partidas_2014_ct.withColumn('away_score', substring('score', 5, 5))\

partidas_2014_ct = partidas_2014_ct.withColumn('home_score', partidas_2014_ct['home_score'].cast(IntegerType()))

partidas_2014_ct = partidas_2014_ct.withColumn('away_score', partidas_2014_ct['away_score'].cast(IntegerType()))

partidas_2014_ct = partidas_2014_ct.withColumn('total_gols', partidas_2014_ct['away_score'] + partidas_2014_ct['home_score'] )

partidas_2014_ct = partidas_2014_ct.withColumn('ano', lit(2014))

time_ganhador = expr(
    """IF(home_score > away_score, home_team, IF(home_score = away_score, 'empate', away_team))"""
)

partidas_2014_ct = partidas_2014_ct.withColumn('result', time_ganhador)

partidas_2014_ct.toPandas()


,_c0,game,round,date,home_team,score,away_team,arena,X,home_score,away_score,total_gols,ano,result
0,1,1,1,20/04/2014 - 18:30,flamengo,0 x 0,goiás,Mané Garrincha - Brasilia - DF,None,0,0,0,2014,empate
1,2,2,1,19/04/2014 - 18:30,fluminense,3 x 0,figueirense,Maracanã - Rio de Janeiro - RJ,None,3,0,3,2014,fluminense
2,3,3,1,20/04/2014 - 16:00,são paulo,3 x 0,botafogo,Morumbi - Sao Paulo - SP,None,3,0,3,2014,são paulo
3,4,4,1,20/04/2014 - 18:30,santos,1 x 1,sport,Vila Belmiro - Santos - SP,None,1,1,2,2014,empate
4,5,5,1,20/04/2014 - 16:00,atletico - pr,1 x 0,grêmio,Orlando Scarpelli - Florianopolis - SC,None,1,0,1,2014,atletico - pr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,376,376,38,07/12/2014 - 17:00,cruzeiro,2 x 1,fluminense,Mineirão - Belo Horizonte - MG,None,2,1,3,2014,cruzeiro
376,377,377,38,07/12/2014 - 17:00,vitória,0 x 1,santos,Manoel Barradas - Salvador - BA,None,0,1,1,2014,santos
377,378,378,38,07/12/2014 - 17:00,grêmio,1 x 1,flamengo,Arena do Grêmio - Porto Alegre - RS,None,1,1,2,2014,empate
378,379,379,38,06/12/2014 - 16:30,figueirense,1 x 2,internacional,Orlando Scarpelli - Florianopolis - SC,None,1,2,3,2014,internacional


## Quantas partidas resultaram em empate?

In [114]:
# -        Quantas partidas resultaram em empate?
count_result = partidas_2014_ct.groupBy("result").agg(count("*")).orderBy("count(1)")

count_result = count_result.withColumn('total_wins', count_result['count(1)'])

count_result = count_result.sort(count_result.total_wins.desc())

count_result = count_result.drop('count(1)')

count_result.show(5)

+-------------+----------+
|       result|total_wins|
+-------------+----------+
|       empate|        92|
|     cruzeiro|        24|
|internacional|        21|
|    são paulo|        20|
|  corinthians|        19|
+-------------+----------+
only showing top 5 rows



In [115]:
sorted_lances_2014_df.show(5)

print(sorted_lances_2014_df.count())

NameError: name 'sorted_lances_2014_df' is not defined

### Lendo todas as partidas

In [8]:
tds_partidas_df = spark.read.csv("/cartola/data/*/*_partidas.csv", header=True)

tds_partidas_df.schema

StructType(List(StructField(_c0,StringType,true),StructField(game,StringType,true),StructField(round,StringType,true),StructField(date,StringType,true),StructField(home_team,StringType,true),StructField(score,StringType,true),StructField(away_team,StringType,true),StructField(arena,StringType,true),StructField(X,StringType,true)))

In [65]:
desc_sorted_partidas_df = tds_partidas_df.sort(tds_partidas_df._c0.desc())

desc_sorted_partidas_df.show(10)

desc_sorted_partidas_df.count()

+---+----+------------------+------------------+----------------+-------------+--------------------+--------------------+----+
|_c0|game|             round|              date|       home_team|        score|           away_team|               arena|   X|
+---+----+------------------+------------------+----------------+-------------+--------------------+--------------------+----+
| 99|  99|                10|16/07/2014 - 22:00|   Criciuma - SC|        3 x 2|     Fluminense - RJ|Heriberto Hulse -...|null|
| 99|  99|                10|02/07/2015 - 19:30|Figueirense - SC|        3 x 1|          Goiás - GO|Orlando Scarpelli...|null|
| 99|  99|                10|25/06/2017 - 16:00|Ponte Preta - SP|        1 x 2|      Palmeiras - SP|Moisés Lucarelli ...|null|
| 99|  99|                10|23/06/2016 - 21:00|  São Paulo - SP|        0 x 0|          Sport - PE|Morumbi - Sao Pau...|null|
| 99|  10|06/06/2018 - 21:00|  Corinthians - SP|           1 x 1|  Santos - SP|Arena Corinthians...|           

2670

In [66]:
asc_sorted_partidas_df = tds_partidas_df.sort(tds_partidas_df._c0.asc())

asc_sorted_partidas_df.show(10)

asc_sorted_partidas_df.count()

+---+----+------------------+------------------+------------------+-----------+--------------------+--------------------+----+
|_c0|game|             round|              date|         home_team|      score|           away_team|               arena|   X|
+---+----+------------------+------------------+------------------+-----------+--------------------+--------------------+----+
|  1|   1|                 1|14/05/2017 - 11:00|   Fluminense - RJ|      3 x 2|         Santos - SP|Maracanã - Rio de...|null|
|  1|   1|                 1|14/05/2016 - 16:00|    Palmeiras - SP|      4 x 0|       Atlético - PR|Allianz Parque - ...|null|
|  1|   1|                 1|09/05/2015 - 18:30|    Palmeiras - SP|      2 x 2|       Atlético - MG|Allianz Parque - ...|null|
|  1|   1|                 1|20/04/2014 - 18:30|     Flamengo - RJ|      0 x 0|          Goiás - GO|Mané Garrincha - ...|null|
|  1|   1|14/04/2018 - 16:00|     Cruzeiro - MG|             0 x 1|Grêmio - RS|Mineirão - Belo H...|           

2670

### Devemos usar os dados de todos os anos?

### Pergunta 1: Quantos registros há na tabela por ano?

In [67]:
# Na tabela partidas:
asc_sorted_partidas_df.count()

2670

#### Podemos contar os registros de cada uma das tabelas que serão criadas ou precisa ser uma única necessariamente?

### Pergunta 2: Quantas equipes únicas mandantes existem?

In [3]:
# Ler tabela times
times_df = spark.read.csv("/cartola/data/times_ids.csv", header=True)
times_df.count()

# sorted_times_df = times_df.sort(times_df.id.desc())
# sorted_times_df.show(20)

47

### Pergunta 3: Quantas vezes as equipes mandantes saíram vitoriosas?

#### Identificado arquivo csv com valores separados por ";" 

### Convert DataFrame Row Object to a new DataFrame

Convert spark DataFrame column of type list to new data frame

In [9]:
df.schema

StructType(List(StructField(atletas,ArrayType(StructType(List(StructField(apelido,StringType,true),StructField(atleta_id,LongType,true),StructField(clube_id,LongType,true),StructField(foto,StringType,true),StructField(jogos_num,LongType,true),StructField(media_num,DoubleType,true),StructField(nome,StringType,true),StructField(pontos_num,DoubleType,true),StructField(posicao_id,LongType,true),StructField(preco_num,DoubleType,true),StructField(rodada_id,LongType,true),StructField(scout,StructType(List(StructField(A,LongType,true),StructField(CA,LongType,true),StructField(CV,LongType,true),StructField(DD,LongType,true),StructField(DP,LongType,true),StructField(FC,LongType,true),StructField(FD,LongType,true),StructField(FF,LongType,true),StructField(FS,LongType,true),StructField(FT,LongType,true),StructField(G,LongType,true),StructField(GC,LongType,true),StructField(GS,LongType,true),StructField(I,LongType,true),StructField(PE,LongType,true),StructField(PP,LongType,true),StructField(RB,Long

In [10]:
atletas_test = df.select("atletas").rdd.flatMap(lambda x: x[0]).collect()


atletas_df = spark.createDataFrame(atletas_test) 
atletas_df.head(3)

# Onde o novo dataframe foi salvo?
# atletas_df.write.json("teste/atletas_teste.json")
atletas_df.schema

StructType(List(StructField(apelido,StringType,true),StructField(atleta_id,LongType,true),StructField(clube_id,LongType,true),StructField(foto,StringType,true),StructField(jogos_num,LongType,true),StructField(media_num,DoubleType,true),StructField(nome,StringType,true),StructField(pontos_num,DoubleType,true),StructField(posicao_id,LongType,true),StructField(preco_num,DoubleType,true),StructField(rodada_id,LongType,true),StructField(scout,StructType(List(StructField(A,LongType,true),StructField(CA,LongType,true),StructField(CV,LongType,true),StructField(DD,LongType,true),StructField(DP,LongType,true),StructField(FC,LongType,true),StructField(FD,LongType,true),StructField(FF,LongType,true),StructField(FS,LongType,true),StructField(FT,LongType,true),StructField(G,LongType,true),StructField(GC,LongType,true),StructField(GS,LongType,true),StructField(I,LongType,true),StructField(PE,LongType,true),StructField(PP,LongType,true),StructField(RB,LongType,true),StructField(SG,LongType,true))),tru

In [11]:
atletas_df.createOrReplaceTempView("tempAtletas") 


# /user/hive/warehouse/teste_tabela_atletas/

In [12]:
sqlContext.sql("create table atletas_hive_test as select * from tempAtletas");


In [13]:
sqlContext.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [14]:
sqlContext.sql("show tables from default").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
| default|atletas_hive_test|      false|
|        |      tempatletas|       true|
+--------+-----------------+-----------+



In [ ]:
CREATE EXTERNAL TABLE atletas (apelido STRING, atleta_id BIGINT)
  STORED AS PARQUET LOCATION '/user/hive/warehouse/teste_tabela_atletas/';

In [84]:
# sqlContext.sql("show databases").show()

sqlContext.sql("CREATE EXTERNAL TABLE atletas (apelido STRING, atleta_id BIGINT) \
               STORED AS PARQUET LOCATION '/user/root/teste/atletas_teste.parquet' ").show()

Py4JJavaError: An error occurred while calling o38.sql.
: java.lang.NoClassDefFoundError: parquet/hadoop/ParquetOutputFormat
	at java.lang.Class.forName0(Native Method)
	at java.lang.Class.forName(Class.java:348)
	at org.apache.spark.util.Utils$.classForName(Utils.scala:238)
	at org.apache.spark.sql.hive.client.HiveClientImpl$.org$apache$spark$sql$hive$client$HiveClientImpl$$toOutputFormat(HiveClientImpl.scala:913)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$toHiveTable$6.apply(HiveClientImpl.scala:947)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$toHiveTable$6.apply(HiveClientImpl.scala:947)
	at scala.Option.map(Option.scala:146)
	at org.apache.spark.sql.hive.client.HiveClientImpl$.toHiveTable(HiveClientImpl.scala:947)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply$mcV$sp(HiveClientImpl.scala:482)
	at org.apache.spark.sql.hive.client.HiveClientImpl$$anonfun$createTable$1.apply(HiveClientImpl.scala:480)
	at org.apache.s

In [ ]:
sc.setProperty("hive.metastore.uris", "thrift://hive-metastore:9083");

In [56]:
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [57]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [61]:
import pyspark
conf = pyspark.SparkConf()

In [62]:
conf.set("spark.executor.uri", "thrift://hive-metastore:9083")

In [64]:
#spark = pyspark.SparkContext(conf=conf)

In [68]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|     default|
+------------+



In [69]:
jdbcDF = spark.read \
    .format("jdbc") \
    .option("driver", "com.facebook.presto.jdbc.PrestoDriver") \
    .option("url", "jdbc:presto://presto:8080/hive/default") \
    .option("user", "hive") \
    .option("dbtable", "pessoas") \
    .load()

Py4JJavaError: An error occurred while calling o749.load.
: java.sql.SQLException: Query failed (#20210421_010659_00001_ddua7): line 3:3: Table hive.default.pessoas does not exist
	at io.prestosql.jdbc.PrestoResultSet.resultsException(PrestoResultSet.java:1894)
	at io.prestosql.jdbc.PrestoResultSet.getColumns(PrestoResultSet.java:1755)
	at io.prestosql.jdbc.PrestoResultSet.<init>(PrestoResultSet.java:125)
	at io.prestosql.jdbc.PrestoStatement.internalExecute(PrestoStatement.java:249)
	at io.prestosql.jdbc.PrestoStatement.execute(PrestoStatement.java:227)
	at io.prestosql.jdbc.PrestoPreparedStatement.executeQuery(PrestoPreparedStatement.java:112)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:61)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:210)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcRelationProvider.createRelation(JdbcRelationProvider.scala:35)
	at org.apache.spark.sql.execution.